# Маркировка 

In [ ]:
#@markdown <br><center><img src='https://upload.wikimedia.org/wikipedia/commons/thumb/d/da/Google_Drive_logo.png/600px-Google_Drive_logo.png' height="50" alt="Gdrive-logo"/></center>
#@markdown <center><h3>Mount GDrive to /content/drive</h3></center><br>
MODE = "MOUNT" #@param ["MOUNT", "UNMOUNT"]
#Mount your Gdrive! 
from google.colab import drive
drive.mount._DEBUG = False
if MODE == "MOUNT":
  drive.mount('/content/drive', force_remount=True)
elif MODE == "UNMOUNT":
  try:
    drive.flush_and_unmount()
  except ValueError:
    pass
  get_ipython().system_raw("rm -rf /root/.config/Google/DriveFS")

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
from sklearn. linear_model import LinearRegression
import matplotlib.pyplot as plt
from pandas.tseries.offsets import DateOffset

In [ ]:
data_febr = pd.read_csv('/content/drive/MyDrive/Vkusvill/1_febr.csv', sep = ';')
data_march = pd.read_csv('/content/drive/MyDrive/Vkusvill/1_march.csv', sep = ';')
data_april = pd.read_csv('/content/drive/MyDrive/Vkusvill/1_april.csv', sep = ';')
data_may = pd.read_csv('/content/drive/MyDrive/Vkusvill/1_may.csv', sep = ';')

In [ ]:
#delete some points for equal length in all files
febr = data_febr.loc[(data_febr['id_tt_cl'] != 15656) & (data_febr['id_tt_cl'] != 16250) & (data_febr['id_tt_cl'] != 11217)]
march = data_march.loc[(data_march['id_tt_cl'] != 11217) & (data_march['id_tt_cl'] != 16250)]
#make sure it works right 
mask = np.isin(febr['id_tt_cl'].unique(), march['id_tt_cl'].unique())
print(mask)

In [ ]:
#febr.groupby('id_tt_cl').date_ch.count()

In [ ]:
#select dates in certain range and then group by id_tt
#but firstly convert dates in the right format
for element in [febr, march, data_april, data_may]:
  element.date_ch = pd.to_datetime(element.date_ch)

In [ ]:
#смотрим, что есть 
#febr.groupby('id_tt_cl').date_ch.agg([min, max])

In [ ]:
#here is the example of february file 
start = pd.Timestamp('2022-02-01')
end = start + DateOffset(weeks=1)
mask = (febr.date_ch >= start) & (febr.date_ch <= end)
febr.loc[mask].groupby('id_tt_cl').BaseSum.sum()

In [ ]:
#we see that one id is missing 
print(np.sort(febr.loc[mask].id_tt_cl.unique()))
print(np.sort(febr['id_tt_cl'].unique()))    

In [ ]:
#this id is 11783, let's consider what's wrong with it 
febr.loc[febr.id_tt_cl == 11783].date_ch.agg([min, max])
#no information of this id in the first week of february

In [ ]:
#check if the information is full of the others
start = pd.Timestamp('2022-02-08')
end = start + DateOffset(weeks=1)
for i in range(3):
  tmp = (febr.date_ch >= start) & (febr.date_ch < end)
  print(len(febr.loc[tmp].id_tt_cl.unique()))
  start = end
  end += DateOffset(weeks=1)
#yes, this part is ok

In [ ]:
#create new table for february
new_data = {'id_tt_cl':np.sort(febr['id_tt_cl'].unique())}
new = pd.DataFrame(data=new_data)

In [ ]:
#index of missing id in first week
ind = np.where(np.sort(febr['id_tt_cl'].unique())== 11783)
print(ind)

In [ ]:
#fill 'manually' for 1st week of february  
start = pd.Timestamp('2022-02-01')
end = start + DateOffset(weeks=1) - DateOffset(days=1)
mask = (febr.date_ch >= start) & (febr.date_ch < end)
for i in range(84):
  if i < 23:
    new.at[i,'sum_week0'] = febr.loc[mask].groupby('id_tt_cl').BaseSum.sum().sort_index().values[i]
  elif i == 23:
    new.at[i,'sum_week0'] = 0 
  elif i > 23:
    new.at[i,'sum_week0'] = febr.loc[mask].groupby('id_tt_cl').BaseSum.sum().sort_index().values[i-1]

In [ ]:
#for other 3 week 
start = pd.Timestamp('2022-02-07')
end = start + DateOffset(weeks=1)
for i in range(3):
  mask = (febr.date_ch >= start) & (febr.date_ch < end)
  new['sum_week'+str(i+1)] = febr.loc[mask].groupby('id_tt_cl').BaseSum.sum().sort_index().values
  start = end
  end += DateOffset(weeks=1)
new.head()

In [ ]:
#since different months may belong to the same week, we join the tables and do the same
data = pd.concat([march, data_april, data_may])

In [ ]:
#check if the information is full again 
start = pd.Timestamp('2022-03-01')
end = start + DateOffset(weeks=1)
for i in range(13):
  tmp = (data.date_ch >= start) & (data.date_ch < end)
  print(len(data.loc[tmp].id_tt_cl.unique())) 
  start = end
  end += DateOffset(weeks=1)
#it's ok

In [ ]:
start = pd.Timestamp('2022-03-01')
end = start + DateOffset(weeks=1) - DateOffset(days=1)
for i in range(3, 16):
  mask = (data.date_ch >= start) & (data.date_ch < end)
  new['sum_week'+str(i+1)] = data.loc[mask].groupby('id_tt_cl').BaseSum.sum().sort_index().values
  start = end
  end += DateOffset(weeks=1)
new.head()

In [ ]:
arr_coef = np.array([0]*84, dtype=float)
x = 10000*np.array(range(2,18)).reshape(-1, 1) 
for i in range(84):
  y = new.iloc[i,1:17].values.reshape(-1, 1)
  reg = LinearRegression()  
  reg.fit(x, y)  
  arr_coef[i] = reg.coef_
new['coef'] = arr_coef
#print(reg.coef_)
#plt.scatter(x, y)
#plt.plot(x, y_pred, color='red')
#plt.show()
new.head()

In [ ]:
new['coef'].describe()

In [ ]:
new.hist(column='coef',bins=25)

In [ ]:
#marking every id 
for i in range(84):
  if new.loc[i, 'coef'] > 1.1:
    new.at[i, 'revenue'] = 'good'
  elif new.loc[i, 'coef'] < 0.9:
    new.at[i,'revenue'] = 'bad'
  else:
    new.at[i,'revenue'] = 'nothing'
new.head()

In [ ]:
new.to_csv('/content/drive/MyDrive/Vkusvill/weeksum_v2.csv', index=False, sep = ';')

# Check's count and average check 

In [ ]:
count_d = {'id_tt_cl':np.sort(febr['id_tt_cl'].unique())}
count_check = pd.DataFrame(data=count_d)
count_check.head()

In [ ]:
start = pd.Timestamp('2022-02-01')
end = start + DateOffset(weeks=1) - DateOffset(days=1)
mask = (febr.date_ch >= start) & (febr.date_ch < end)
febr.loc[mask].groupby(['id_tt_cl', 'CheckUID']).CheckUID.count()

In [ ]:
start = pd.Timestamp('2022-02-01')
end = start + DateOffset(weeks=1) - DateOffset(days=1)
mask = (febr.date_ch >= start) & (febr.date_ch < end)
for i in range(84):
  if i < 23:
    count_check.at[i,'count_week0'] = len(febr.loc[mask].groupby('id_tt_cl').CheckUID.unique().sort_index().values[i])
  elif i == 23:
    count_check.at[i,'count_week0'] = 0 
  elif i > 23:
    count_check.at[i,'count_week0'] = len(febr.loc[mask].groupby('id_tt_cl').CheckUID.unique().sort_index().values[i-1])

In [ ]:
start = pd.Timestamp('2022-02-07')
end = start + DateOffset(weeks=1)
for i in range(3):
  mask = (febr.date_ch >= start) & (febr.date_ch < end)
  arr = np.array(len(febr.loc[mask].groupby('id_tt_cl').CheckUID.unique().sort_index().values[i]) for i in range(84))
  count_check['count_week'+str(i+1)] = arr
  start = end
  end += DateOffset(weeks=1)
count_check.head()

In [ ]:
start = pd.Timestamp('2022-03-01')
end = start + DateOffset(weeks=1) - DateOffset(days=1)
for i in range(3, 16):
  mask = (data.date_ch >= start) & (data.date_ch < end)
  arr = np.array(len(data.loc[mask].groupby('id_tt_cl').CheckUID.unique().sort_index().values[i]) for i in range(84))
  count_check['count_week'+str(i+1)] = arr
  start = end
  end += DateOffset(weeks=1)
count_check.head()

In [ ]:
count_check.to_csv('/content/drive/MyDrive/Vkusvill/count.csv', index=False)

In [ ]:
count_check = pd.read_csv('/content/drive/MyDrive/Vkusvill/count.csv')

In [ ]:
new = pd.read_csv('/content/drive/MyDrive/Vkusvill/weeksum_v2.csv', sep = ';')

In [ ]:
average_check_d = {'id_tt_cl':np.sort(count_check['id_tt_cl'].unique())} 
average_check = pd.DataFrame(data=average_check_d)

In [ ]:
for i in range(17):
  average_check['av_check_week'+str(i)] = np.divide(new['sum_week'+str(i)].sort_index().values, count_check['count_week'+str(i)].sort_index().values)
average_check.head()

In [ ]:
average_check.to_csv('/content/drive/MyDrive/Vkusvill/average.csv', index=False)